# Create Variable Groups for CORE pipelines

For each application
- Get the Environment Variables
- Extract the Secrets
- Create Variable Group
- Add Variables

## Imports

In [ ]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
from cloudfoundry_client.client import CloudFoundryClient
import re
import pprint

## Get Applications Data

Here I connect to Cloud Foundry and load all the environment variables for every app in the space into the variable `applications_variables`. The Space is defined by the GUID.

Requires a username and password.

In [ ]:
target_endpoint = 'https://api.cf.us10.hana.ondemand.com'
username = ''
password = ''

client = CloudFoundryClient(target_endpoint)
# init with user credentials
client.init_with_user_credentials(username, password)

applications_variables = {}

for app in client.v2.apps.list(**{'space_guid': 'b0a49358-1099-43e9-8da4-67c3395f076b'}):
    applications_variables[app['entity']['name']] = app['entity']['environment_json']


## Extract the Secrets

This pulls out all the secret variables as defined below and stores them in `applications_secrets`.

In [ ]:
applications_secrets = {}

def convert(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    return s2.replace('.', '_').replace('__', '_').replace('mx_', '')


for app_name in applications_variables:
    envars = applications_variables[app_name]
    secrets = {}
    for var in envars:
        value = envars[var]
        if 'key' in var.lower() \
           or 'secret' in var.lower() \
           or 'password' in var.lower() \
           or 'pass' in var.lower() \
           or 'sec' in var.lower():
            secrets[convert(var)] = value
    applications_secrets[app_name] = secrets

pprint.pprint(applications_secrets)

## Roadside Variable Group

This creates a variable group for Roadside Specificly with all the secrets. 

Requires a PAT (personal access token)

In [ ]:
# Fill in with your personal access token and org URL
personal_access_token = ''
organization_url = 'https://dev.azure.com/lovestfs'

# Create a connection to the org
credentials = BasicAuthentication('', personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)

# Get a client (the "core" client provides access to projects, teams, etc)
task_agent_client = connection.clients_v6_0.get_task_agent_client()

secrets = {}
app = 'Roadside-Stage'
for secret in applications_secrets[app]:
    value = applications_secrets[app][secret]
    secrets[secret] = {'isSecret': True,
                        'value': value}

request_data = {'description': 'Production Secrets for ' + app,
                'name': 'Roadside-Prod-Secrets',
                'type': 'Vsts',
                'variableGroupProjectReferences': [{'name': 'Roadside-Prod-Secrets', 'projectReference': {'name': 'MendixDevelopment'}}],
                'variables': secrets}
add_variable_group_response = task_agent_client.add_variable_group(request_data)
pprint.pprint(add_variable_group_response)

# cf push parameters

This outputs the cf push parameters by application

In [ ]:
for app in applications_secrets:
    secret_vars = applications_secrets[app]
    parameters = '-p "_Build/drop/prod.yml'
    for key in secret_vars:
        value = secret_vars[key]
        parameters += f' --var {key}=$({key})'
    print(f'{app}: {parameters}\n\n')

## Create Variable Groups

Creates all the other variable Groups

In [ ]:
# Fill in with your personal access token and org URL
personal_access_token = '2p7jquww7qdjxwnlzrnhmgdxn5lu4kqseq2dt6i5urfyvqramurq'
organization_url = 'https://dev.azure.com/lovestfs'

# Create a connection to the org
credentials = BasicAuthentication('', personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)

# Get a client (the "core" client provides access to projects, teams, etc)
task_agent_client = connection.clients_v6_0.get_task_agent_client()

# get_variable_group_response = task_agent_client.get_variable_group('MendixDevelopment', '80')
# pprint.pprint(get_variable_group_response.name)

for app in applications_secrets:
    secrets = {}
    for secret in applications_secrets[app]:
        value = applications_secrets[app][secret]
        secrets[secret] = {'isSecret': True,
                           'value': value}
    
    request_data = {'description': 'Production Secrets for ' + app,
                    'name': app + '-Prod-Secrets',
                    'type': 'Vsts',
                    'variableGroupProjectReferences': [{'name': app + '-Prod-Secrets', 'projectReference': {'name': 'MendixDevelopment'}}],
                    'variables': secrets}
    add_variable_group_response = task_agent_client.add_variable_group(request_data)
    pprint.pprint(add_variable_group_response)